In [1]:
import numpy as np
import pandas as pd
import plotly.express as px
import seaborn as sns
import holoviews as hv
import hvplot.pandas

hv.extension('bokeh')

huel_start_date = pd.Timestamp(2024, 9, 5)
end_date = pd.Timestamp(2024, 9, 12)
start_date = huel_start_date - pd.Timedelta(days=30*3) # 3 months before the start of the challenge

def pace_to_td(pace: str):
    min, sec = pace.split(':')

    return pd.Timedelta(minutes=int(min), seconds=int(sec))

def pace_to_kmh(pace: str):
    min, sec = pace.split(':')
    return 60 / (int(min) + int(sec) / 60)

def pace_to_sorta_mpkm(pace: str):
    min, sec = pace.split(':')
    return int(min) + int(sec) / 60


activity= pd.read_csv('huel/activities.csv')
activity = (
    activity
    .loc[lambda x: x['Activity Type'] == 'Running']
    .assign(
        date=lambda x: pd.to_datetime(x['Date']),
        avg_pace=lambda x: x['Avg Pace'].map(pace_to_td),
        avg_pace_kmh=lambda x: x['Avg Pace'].map(pace_to_kmh),
        avg_pace_smpkm=lambda x: x['Avg Pace'].map(pace_to_sorta_mpkm),
        during_challenge = lambda x: x.date >= huel_start_date,
        group=lambda x: x.during_challenge.map({True: 'Huel', False: 'Baseline'}),
        group_jitter=lambda x: x.during_challenge.astype(float) + np.random.normal(0, 0.035, len(x))
    )
    .rename(columns={
        'avg_pace': 'Average Pace (mins/km)',
        'avg_pace_kmh': 'Average Pace (kmh)',
        'avg_pace_smpkm': 'Average Pace (fractional mins/km)',
    })
    .loc[lambda x: x.date >= start_date]
    .loc[lambda x: x.date <= end_date]
    .sort_values('date', )
)

# activity.hvplot(kind='scatter', x='date', y='Average Pace (mins/km)', by='group', cmap='viridis', title='Running Pace Over Time').opts(legend_position='top_left')

fig = px.box(activity, x='group', y='Average Pace (fractional mins/km)', color='group', points='all', title='Average Pace')


def fig_to_html(fig, name):
    fig.update_layout(showlegend=False)
    with open(f'../public/iframes/huel_{name}.html', 'w') as f:
        f.write(fig.to_html(include_plotlyjs='cdn'))

fig_to_html(fig, 'avg_pace')

fig 

In [2]:
# activity.hvplot(kind='box', x='group', by='group', y='Average Pace (kmh)', cmap='viridis', title='Average Pace (km/h)').opts(legend_position='top_left')

In [3]:
# activity.hvplot(kind='box', x='group', by='group', y='Distance', cmap='viridis', title='Distance run').opts(legend_position='top_left')

fig = px.box(activity, x='group', y='Distance', color='group', points='all', title='Average Distance (kilometers)')

fig_to_html(fig, 'avg_dist')

fig

In [4]:
# activity.hvplot(kind='box', x='group', by='group', y='Avg HR', cmap='viridis', title='Average Heart Rate During Exercise').opts(legend_position='top_left')
px.box(activity, x='group', y='Avg HR', color='group', points='all', title='Average Heart Rate During Exercise')

In [5]:
sleep = (
    pd.read_csv('huel/sleep.csv')
    .assign(
        date=lambda x: pd.to_datetime(x['Sleep Score 4 Weeks'] + ' 2024'),
        during_challenge = lambda x: x.date >= huel_start_date,

        group=lambda x: x.during_challenge.map({True: 'Huel', False: 'Baseline'}),
    )
    .sort_values('date')
)


In [6]:
sleep

,Sleep Score 4 Weeks,Score,Resting Heart Rate,Body Battery,Pulse Ox,Respiration,HRV Status,Quality,Duration,Bedtime,Wake Time,date,during_challenge,group
27,15 Aug,66,53,68,--,13,54,Fair,7h 40min,23:14,7:39,2024-08-15,False,Baseline
26,16 Aug,41,56,28,--,14,52,Poor,5h 51min,2:19,8:46,2024-08-16,False,Baseline
25,17 Aug,85,50,87,--,12,54,Good,9h 14min,0:32,10:00,2024-08-17,False,Baseline
24,18 Aug,92,48,68,--,12,56,Excellent,8h 37min,0:19,8:57,2024-08-18,False,Baseline
23,19 Aug,75,49,--,--,13,56,Fair,7h 43min,23:24,7:31,2024-08-19,False,Baseline
22,20 Aug,81,51,72,--,13,55,Good,7h 45min,23:22,7:26,2024-08-20,False,Baseline
21,21 Aug,83,51,71,--,12,56,Good,7h 23min,0:52,8:21,2024-08-21,False,Baseline
20,22 Aug,73,50,73,--,13,57,Fair,8h 9min,23:29,8:17,2024-08-22,False,Baseline
19,23 Aug,84,50,--,--,12,61,Good,6h 54min,0:40,7:55,2024-08-23,False,Baseline
18,24 Aug,70,51,57,--,13,60,Fair,7h 48min,23:55,8:31,2024-08-24,False,Baseline


In [7]:
# sleep.hvplot(kind='box', x='group', by='group', y='Score', cmap='viridis', title='Average Sleep Score')
fig = px.box(sleep, x='group', y='Score', color='group', points='all', title='Average Sleep Score')

fig_to_html(fig, 'sleep')
fig

In [8]:
# This is 'body battery charge during sleep' - i don't care about this measure, i thought it was just body battery
 
# def tryfloat(x):
#     try:
#         return float(x)
#     except ValueError:
#         return float('nan')

# sleep['Body Battery'] = sleep['Body Battery'].map(tryfloat)

# fig = px.box(sleep, x='group', y='Body Battery', color='group', points='all', title='Average Body Battery')

# fig_to_html(fig, 'bb')
# fig

# Load up data from Garmin export

In [9]:
from pathlib import Path

folder = Path('/Users/george/Downloads/fcf1aca2-53f6-4c22-a772-020c3a72b296_1/DI_CONNECT/DI-Connect-Aggregator/')

folder.exists()
files = folder.glob('UDSFile_*.json')


uds = []
for file in files:
    uds.append(pd.read_json(file))

uds = pd.concat(uds)
uds.info()


<class 'pandas.core.frame.DataFrame'>
Index: 1661 entries, 0 to 98
Data columns (total 58 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   userProfilePK                         1661 non-null   int64  
 1   calendarDate                          1661 non-null   object 
 2   uuid                                  1661 non-null   object 
 3   durationInMilliseconds                1661 non-null   int64  
 4   totalKilocalories                     1661 non-null   int64  
 5   activeKilocalories                    1661 non-null   int64  
 6   bmrKilocalories                       1661 non-null   int64  
 7   wellnessKilocalories                  1661 non-null   int64  
 8   remainingKilocalories                 1661 non-null   int64  
 9   wellnessTotalKilocalories             1661 non-null   int64  
 10  wellnessActiveKilocalories            1661 non-null   int64  
 11  totalSteps              

In [10]:
# # pd.DataFrame()
# bb.bodyBatteryStatList.tolist()
bb = uds['bodyBattery'].str['bodyBatteryStatList'].tolist()

# Flatten the list of lists
flat = []
for sublist in bb:
    if isinstance(sublist, float):
        continue
    for item in sublist:
        flat.append(item)
bb = (
    pd.DataFrame(flat).rename(columns={'statsValue': 'Body Battery'})
    .assign(statTimestamp=lambda x: pd.to_datetime(x.statTimestamp),
           during_challenge = lambda x: x.statTimestamp >= huel_start_date,
              group=lambda x: x.during_challenge.map({True: 'Huel', False: 'Baseline'}), 
            )
    .loc[lambda x: x.statTimestamp >= start_date]
    .loc[lambda x: x.statTimestamp <= end_date]

)

# Daily high 
bb_high = (
    bb
    .loc[lambda x: x.bodyBatteryStatType == 'HIGHEST']
)
fig = px.box(bb_high, x='group', y='Body Battery', color='group', points='all', title='Daily High Body Battery')

fig_to_html(fig, 'bb_high')
fig

In [12]:
# Daily high 
bb_low = (
    bb
    .loc[lambda x: x.bodyBatteryStatType == 'LOWEST']
)
fig = px.box(bb_low, x='group', y='Body Battery', color='group', points='all', title='Daily Low Body Battery')

fig_to_html(fig, 'bb_low')
fig